In [1]:
from librosa.core.convert import midi_to_note
import os
import pathlib
import librosa
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

def path_to_audiofiles(dir_folder):
    list_of_audio = []
    for file in os.listdir(dir_folder):
        if file.endswith(".wav"):
            directory = "%s%s" % (dir_folder, file)
            list_of_audio.append(directory)
    return list_of_audio

def min_max(x, axis=None): # do min_max normalization ?->https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html
    min = 0
    max = 154.28702 
    # max = 137.74599
    x = np.abs(x)
    result = (x-min)/(max-min)
    return result


def audio_stft_crop(filepath,crop_position=224,n_fft = 512,hop_length = 256):
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    stft = min_max(stft)
    stft = stft[:crop_position,:]
    stft = cv2.cvtColor(stft, cv2.COLOR_GRAY2RGB)
    return stft
  except Exception:
    return

In [2]:
import os
import torch
from PIL import Image
import numpy as np
from tqdm import tqdm
from keras.utils import np_utils

def make_dataset(cp, path="/content/drive/MyDrive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s"):
  x = []
  y = []
  clss = [1, 2, 3, 4]
  for cls in clss:
    for a in tqdm(path_to_audiofiles(path + '/' + 'Q' + str(cls) + '/')):
      im = audio_stft_crop(a, crop_position=cp)
      x.append(im)
      y.append(cls-1)
  x = np.array(x)
  y = np.array(y)
  y = np_utils.to_categorical(y)
  x = tf.convert_to_tensor(x)
  y = tf.convert_to_tensor(y)
  return x,y

# Model

In [3]:
#@title Model Def1 {display-mode: "form"}
import math
from keras import layers
import copy
import re
"""EfficientNet V1 and V2 model configs."""

#################### EfficientNet V1 configs ####################
v1_b0_block_str = [
        'r1_k3_s1_e1_i32_o16_se0.25',
        'r2_k3_s2_e6_i16_o24_se0.25',
        'r2_k5_s2_e6_i24_o40_se0.25',
        'r3_k3_s2_e6_i40_o80_se0.25',
        'r3_k5_s1_e6_i80_o112_se0.25',
        'r4_k5_s2_e6_i112_o192_se0.25',
        'r1_k3_s1_e6_i192_o320_se0.25',
]

#################### EfficientNet V2 configs ####################
v2_base_block = [    # The baseline config for v2 models.
        'r1_k3_s1_e1_i32_o16_c1',
        'r2_k3_s2_e4_i16_o32_c1',
        'r2_k3_s2_e4_i32_o48_c1',
        'r3_k3_s2_e4_i48_o96_se0.25',
        'r5_k3_s1_e6_i96_o112_se0.25',
        'r8_k3_s2_e6_i112_o192_se0.25',
]

v2_s_block = [    # about base * (width1.4, depth1.8)
        'r2_k3_s1_e1_i24_o24_c1',
        'r4_k3_s2_e4_i24_o48_c1',
        'r4_k3_s2_e4_i48_o64_c1',
        'r6_k3_s2_e4_i64_o128_se0.25',
        'r9_k3_s1_e6_i128_o160_se0.25',
        'r15_k3_s2_e6_i160_o256_se0.25',
]


v2_m_block = [    # about base * (width1.6, depth2.2)
        'r3_k3_s1_e1_i24_o24_c1',
        'r5_k3_s2_e4_i24_o48_c1',
        'r5_k3_s2_e4_i48_o80_c1',
        'r7_k3_s2_e4_i80_o160_se0.25',
        'r14_k3_s1_e6_i160_o176_se0.25',
        'r18_k3_s2_e6_i176_o304_se0.25',
        'r5_k3_s1_e6_i304_o512_se0.25',
]


v2_l_block = [    # about base * (width2.0, depth3.1)
        'r4_k3_s1_e1_i32_o32_c1',
        'r7_k3_s2_e4_i32_o64_c1',
        'r7_k3_s2_e4_i64_o96_c1',
        'r10_k3_s2_e4_i96_o192_se0.25',
        'r19_k3_s1_e6_i192_o224_se0.25',
        'r25_k3_s2_e6_i224_o384_se0.25',
        'r7_k3_s1_e6_i384_o640_se0.25',
]

v2_xl_block = [    # only for 21k pretraining.
        'r4_k3_s1_e1_i32_o32_c1',
        'r8_k3_s2_e4_i32_o64_c1',
        'r8_k3_s2_e4_i64_o96_c1',
        'r16_k3_s2_e4_i96_o192_se0.25',
        'r24_k3_s1_e6_i192_o256_se0.25',
        'r32_k3_s2_e6_i256_o512_se0.25',
        'r8_k3_s1_e6_i512_o640_se0.25',
]
efficientnet_params = {
        # (block, width, depth, train_size, eval_size, dropout, randaug, mixup, aug)
        'efficientnetv2-s':    # 83.9% @ 22M
                (v2_s_block, 1.0, 1.0, 300, 384, 0.2, 10, 0, 'randaug'),
        'efficientnetv2-m':    # 85.2% @ 54M
                (v2_m_block, 1.0, 1.0, 384, 480, 0.3, 15, 0.2, 'randaug'),
        'efficientnetv2-l':    # 85.7% @ 120M
                (v2_l_block, 1.0, 1.0, 384, 480, 0.4, 20, 0.5, 'randaug'),

        'efficientnetv2-xl':
                (v2_xl_block, 1.0, 1.0, 384, 512, 0.4, 20, 0.5, 'randaug'),

        # For fair comparison to EfficientNetV1, using the same scaling and autoaug.
        'efficientnetv2-b0':    # 78.7% @ 7M params
                (v2_base_block, 1.0, 1.0, 192, 224, 0.2, 0, 0, 'effnetv1_autoaug'),
        'efficientnetv2-b1':    # 79.8% @ 8M params
                (v2_base_block, 1.0, 1.1, 192, 240, 0.2, 0, 0, 'effnetv1_autoaug'),
        'efficientnetv2-b2':    # 80.5% @ 10M params
                (v2_base_block, 1.1, 1.2, 208, 260, 0.3, 0, 0, 'effnetv1_autoaug'),
        'efficientnetv2-b3':    # 82.1% @ 14M params
                (v2_base_block, 1.2, 1.4, 240, 300, 0.3, 0, 0, 'effnetv1_autoaug'),

        'efficientnet-b0': (v1_b0_block_str, 1.0, 1.0, 224, 224, 0.2),
        'efficientnet-b1': (v1_b0_block_str, 1.0, 1.1, 240, 240, 0.2),
        'efficientnet-b2': (v1_b0_block_str, 1.1, 1.2, 260, 260, 0.3),
        'efficientnet-b3': (v1_b0_block_str, 1.2, 1.4, 300, 300, 0.3),
        'efficientnet-b4': (v1_b0_block_str, 1.4, 1.8, 380, 380, 0.4),
        'efficientnet-b5': (v1_b0_block_str, 1.6, 2.2, 456, 456, 0.4),
        'efficientnet-b6': (v1_b0_block_str, 1.8, 2.6, 528, 528, 0.5),
        'efficientnet-b7': (v1_b0_block_str, 2.0, 3.1, 600, 600, 0.5),
        'efficientnet-b8': (v1_b0_block_str, 2.2, 3.6, 672, 672, 0.5),
        'efficientnet-l2': (v1_b0_block_str, 4.3, 5.3, 800, 800, 0.5),
}



class Struct:
        def __init__(self, **entries):
                self.__dict__.update(entries)

def make_config(model_name):
    def round_filters(filters, multiplier, divisor, min_depth=None, skip=False):
        """Round number of filters based on depth multiplier."""
        if skip or not multiplier:
            return filters

        filters = filters*multiplier
        min_depth = min_depth or divisor
        new_filters = max(min_depth, int(filters + divisor / 2) // divisor * divisor)
        return int(new_filters)


    def round_repeats(repeats, multiplier, skip=False):
        """Round number of filters based on depth multiplier."""
        if skip or not multiplier:
            return repeats
        return int(math.ceil(multiplier * repeats))    

    cfg = efficientnet_params[model_name]
    width_coefficient = cfg[1]
    depth_coefficient = cfg[2] 

    block_args_list = []
    for block_string in cfg[0]:
        ops = block_string.split('_')
        options = {}
        for op in ops:
            splits = re.split(r'(\d.*)', op)
            if len(splits) >= 2:
                key, value = splits[:2]
                options[key] = value

        input_filters = round_filters(int(options['i']), width_coefficient, 8)
        output_filters = round_filters(int(options['o']), width_coefficient, 8)

        block0_args =    Struct(
                kernel_size=int(options['k']),
                input_filters=input_filters,
                output_filters=output_filters,
                expand_ratio=int(options['e']),
                se_ratio=float(options['se']) if 'se' in options else 0.0,
                strides=int(options['s']),
                conv_type=int(options['c']) if 'c' in options else 0,
        )

        block_args_list.append(block0_args)
        repeats = round_repeats(int(options['r']), depth_coefficient)
        for _ in range(repeats - 1):
            block_args = copy.deepcopy(block0_args)
            block_args.input_filters=int(output_filters)
            block_args.strides=int(1)
            block_args_list.append(block_args)

    mconfig = Struct(
        name=model_name,
        block_args_list = block_args_list,
        train_size = cfg[3],
        eval_size = cfg[4],
        dropout_rate = cfg[5],
        num_classes = 4,  # changed 1000 to 4
        feature_size = round_filters( 1280, width_coefficient, 8),
        depth_divisor = 8,
        bn_momentum = 0.9,
        bn_epsilon = 1e-3,
        min_depth = 8,
        survival_prob = 0.8,
        weight_decay = 0.0,
        stem_stride=2,

    )
    return mconfig

In [4]:
#@title Model Def2 {display-mode: "form"}
def conv_kernel_initializer(shape, dtype=None, partition_info=None):
    del partition_info
    kernel_height, kernel_width, _, out_filters = shape
    fan_out = int(kernel_height * kernel_width * out_filters)
    return tf.random.normal(
            shape, mean=0.0, stddev=np.sqrt(2.0 / fan_out), dtype=dtype)

def dense_kernel_initializer(shape, dtype=None, partition_info=None):
    del partition_info
    init_range = 1.0 / np.sqrt(shape[1])
    return tf.random.uniform(shape, -init_range, init_range, dtype=dtype)

def create_efficientnet(config, input_shape=(None,None,3) , name=None, verbose=0):
    mconfig = config
    BN_KWARGS = {'momentum':mconfig.bn_momentum, 'epsilon':mconfig.bn_epsilon}
    CONV_KWARGS = {'use_bias':False, 'padding':'same', 
                    'kernel_initializer': conv_kernel_initializer, 
                    'kernel_regularizer': tf.keras.regularizers.l2(mconfig.weight_decay)}

    def activation(fn='silu', name=None):
        if fn=='sigmoid':
            return layers.Lambda( lambda y: tf.sigmoid(y), name=name)
        return layers.Lambda( lambda y: tf.nn.silu(y), name=name)

    def conv_bn_act(x, filters, kernel, stride, prefix):
        x = layers.Conv2D( filters, kernel, stride, name=prefix+'_conv2d',    **CONV_KWARGS)(x)
        x = layers.BatchNormalization(name=prefix+'_bnorm', **BN_KWARGS)(x)
        x = activation(name=prefix+'_act')(x)
        return x

    def se(inputs, se_filters, filters, prefix):
        SE_CONV_KWARGS = {'use_bias':True, 'padding':'same', 
                        'kernel_initializer': conv_kernel_initializer,
                        'kernel_regularizer': tf.keras.regularizers.l2(mconfig.weight_decay)}
        x = layers.Lambda(lambda y:tf.reduce_mean(y, [1, 2], keepdims=True), name=prefix+'_se_mean')(inputs)
        x = layers.Conv2D(se_filters, 1, 1, name=prefix+'_se_conv2d_s', **SE_CONV_KWARGS)(x)
        x = activation(name=prefix+'_se_act_s')(x)
        x = layers.Conv2D(filters, 1, 1, name=prefix+'_se_conv2d_e', **SE_CONV_KWARGS)(x)
        x = activation('sigmoid', name=prefix+'_se_act_e')(x)
        x = layers.Multiply(name=prefix+'_se_mult')( [x ,inputs ])
        return x

    def mb_conv(inputs, block_args, prefix):
        filters = block_args.input_filters * block_args.expand_ratio
        se_filters = int(block_args.input_filters*block_args.se_ratio)
        x = inputs
        if block_args.expand_ratio != 1:
            x = layers.Conv2D( filters, 1, 1, name=prefix+'_conv2d_expand', **CONV_KWARGS)(x)
            x = layers.BatchNormalization(name=prefix+'_bnorm_expand', **BN_KWARGS)(x)
            x = activation(name=prefix+'_act_expand')(x)

        x = layers.DepthwiseConv2D( block_args.kernel_size, block_args.strides, name=prefix+'_dwconv2d', **CONV_KWARGS)(x)
        x = layers.BatchNormalization(name=prefix+'_bnorm', **BN_KWARGS)(x)
        x = activation(name=prefix+'_act')(x)
        if se_filters!=0:
            x = se(x, se_filters, filters, prefix=prefix)
        x = layers.Conv2D( block_args.output_filters, 1, 1, name=prefix+'_conv2d_proj', **CONV_KWARGS)(x)
        x = layers.BatchNormalization(name=prefix+'_bnorm_proj', **BN_KWARGS)(x)

        return x

    def fused_mb_conv(inputs, block_args, prefix):
        filters = block_args.input_filters * block_args.expand_ratio
        se_filters = int(block_args.input_filters*block_args.se_ratio)
        x = inputs
        if block_args.expand_ratio != 1:
            x = layers.Conv2D( filters, block_args.kernel_size, block_args.strides, 
                name=prefix+'_conv2d_expand', **CONV_KWARGS)(x)
            x = layers.BatchNormalization(name=prefix+'_bnorm_expand', **BN_KWARGS)(x)
            x = activation(name=prefix+'_act_expand')(x)
            if se_filters!=0:
                x = se(x, se_filters, filters, prefix=prefix)
            x = layers.Conv2D( block_args.output_filters, 1, 1, 
                name = prefix+'_conv2d', **CONV_KWARGS )(x)
            x = layers.BatchNormalization(name=prefix+'_bnorm', **BN_KWARGS)(x)
        else:
            if se_filters!=0:
                x = se(x, se_filters, filters, prefix=prefix)
            x = layers.Conv2D( block_args.output_filters, block_args.kernel_size, block_args.strides,
                name = prefix+'_conv2d', **CONV_KWARGS )(x)
            x = layers.BatchNormalization(name=prefix+'_bnorm', **BN_KWARGS)(x)
            x = activation(name=prefix+'_act')(x) # add act if no expansion.

        return x

    # Input
    x = input = tf.keras.Input(shape=input_shape, name='input')


    # Stem
    stem_filters = mconfig.block_args_list[0].input_filters
    x = conv_bn_act(x, stem_filters , 3, mconfig.stem_stride, prefix='stem')
    if verbose!=0:
        print('stem', 3, stem_filters, mconfig.stem_stride )

    # Blocks
    for i, block_args in enumerate(mconfig.block_args_list):
        shortcut = x
        if block_args.conv_type==0:
            prefix = f'block_{i}'
            x = mb_conv(x, block_args, prefix)
        else:
            prefix = f'block_{i}_fused'
            x = fused_mb_conv(x, block_args, prefix)

        survival_prob = mconfig.survival_prob
        if block_args.strides == 1 and block_args.input_filters==block_args.output_filters:
            if survival_prob:
                # StochasticDepth
                drop_rate = 1.0 - survival_prob
                drop_rate = drop_rate * float(i+1) / len(mconfig.block_args_list)
                survival_prob = 1.0 - drop_rate
                x = layers.Dropout(rate=drop_rate, noise_shape=(None,1,1,1), name=prefix+'_stochdepth')(x)
            x = layers.Add(name=prefix+'_add')([shortcut,x])
        else:
            survival_prob = None

        if verbose!=0:
            print(i, block_args.kernel_size,block_args.input_filters, block_args.output_filters, 
                block_args.strides, block_args.expand_ratio, survival_prob    )

    # Head
    x = conv_bn_act( x,mconfig.feature_size , 1, 1, prefix='head')
    x = layers.GlobalAveragePooling2D(name='head_gap')(x)
    if mconfig.dropout_rate > 0:
        x = layers.Dropout(mconfig.dropout_rate,name='head_dropout')(x)
    output = layers.Dense(mconfig.num_classes,
            kernel_initializer = dense_kernel_initializer,
            kernel_regularizer = tf.keras.regularizers.l2(mconfig.weight_decay),
            use_bias = True, name='head_fc')(x)
    if verbose!=0:
        print('head', mconfig.feature_size, mconfig.num_classes, mconfig.dropout_rate    )

    model = tf.keras.Model(inputs=[input], outputs=[output], name=name)
    if verbose!=0:
        print('params', model.count_params())
    return model


#Cropped (0.2)

In [5]:
x,y = make_dataset(102)
print(x.__sizeof__())
print(y.__sizeof__())

100%|██████████| 2800/2800 [02:40<00:00, 17.40it/s]


152
152


In [6]:
!nvidia-smi

Sat Aug 27 08:46:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    34W / 250W |   4485MiB / 16280MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# whether to finetune the whole model or just the top layer.
import numpy as np
import tensorflow as tf
num_epochs = 1 #@param {type:"integer"}

tf.keras.backend.clear_session()
config = make_config("efficientnetv2-b0")
model = create_efficientnet(config, input_shape=(x[0].shape[0], x[0].shape[1],3))

model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])


hist = model.fit(
    x,y,validation_split=0.2,
    epochs=num_epochs).history

280/280 [==============================] - 37s 72ms/step - loss: 1.7434 - accuracy: 0.4345 - val_loss: 2.3652 - val_accuracy: 0.0000e+00


In [8]:
print("[cropped 0.2]")
!nvidia-smi

[cropped 0.2]
Sat Aug 27 08:47:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |   8883MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                          

In [9]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
import tensorflow as tf
tf.keras.backend.clear_session()